# 6-1. 関数プログラミング
関数プログラミングについて説明します。

参考
- https://docs.python.org/ja/3/howto/functional.html
- https://docs.python.org/ja/3/library/functions.html
- https://docs.python.org/ja/3/library/itertools.html
- https://docs.python.org/ja/3/library/functools.html

**関数プログラミング**（**functional programming**）とは、プログラムを（数学的な）関数の合成で記述するプログラミングスタイルです。
処理を操作列と考えて命令的に記述するのではなく、処理をデータ変換を行う関数に分解して記述します。
これをPythonで行うときに重要になるのは、高階関数とイテレータです。
したがって、Pythonにおける関数プログラミングとは、高階関数とイテレータを使いこなすことだと考えても、ほぼ差し支えありません。

## 高階関数

**高階関数** （**higher-order function**）とは、値として関数を受け取ったり返したりする関数のことです。
Pythonにおける関数はオブジェクトなので、定義した関数をそのまま渡したり返したりすることができます。

In [ ]:
def inc(x):
    return x+1

def twice(f, x):
    return f(f(x))

def genfunc():
    return inc

twice(genfunc(), 0)

ここで、`twice()`は関数を受け取り、`genfunc()`は関数を返しているので、どちらも高階関数です。

組込み関数などのよく使われる関数には、関数を受け取る高階関数が多いです。
そのような高階関数を使うときには、上に示した`inc()`のように、小さい関数を渡したくなることがよくあります。
この時に便利なのが、**ラムダ式**（または**無名関数**）です。例えば、
```Python
lambda x: x+1
```
は、`inc()`と等価な関数オブジェクトと返します。一般に、
```Python
f = lambda 引数: 式
```
は
```Python
def f(引数):
    return 式
```
と同等です。

ラムダ式は、`def`記法による関数定義に比べて記述に制限が加わりますが、関数呼出しの引数の位置に関数定義を記述できるという利点があります。
例えば、`twice(inc, 0)`の代わりに`twice(lambda x: x+1, 0)`と呼び出すなら、わざわざ`inc()`を定義しなくて済みます。
このように、ラムダ式を有効活用すると、全体のコードが簡潔で読みやすくなります。

### sorted
2-2で、整列（ソート）されたリストを返す関数**`sorted()`**を導入しました。

In [ ]:
sorted([1,3,-2,0])

実は、`sorted()`は`key`引数に関数を取れる高階関数です。
`key`引数は、各要素を比較に使われる値に変換する関数を取ります。
例えば、絶対値の昇順で整列したい場合、絶対値関数`abs()`を`key`引数に渡せばよいです。

In [ ]:
sorted([1,3,-2,0], key=abs)

#### 練習
文字列のキーと数値の値のペアのリスト`ls`があるとする。例えば、`ls = [('A', 1), ('B', 3), ('C', -1), ('D', 0)]`。
このリスト`ls`を、値の降順で整列するように、`sorted()`を呼び出せ。

### max, min

組込み関数**`max()`**と**`min()`**は、それぞれ最大の要素と最小の要素を返す関数です。

In [ ]:
max([1,3,-2,0])

In [ ]:
min([1,3,-2,0])

`sorted()`と同様に、どちらも`key`引数に、比較に使われる値に変換する関数を取れます。
したがって、例えば`abs()`を渡せば、絶対値が最大と最小となる要素を返します。

In [ ]:
max([1,-3,-2,0], key=abs)

In [ ]:
min([1,-3,-2,0], key=abs)

#### 練習
リスト（例えば`[1,3,-2,0]`）の最小の要素を返すように、`max()`を用いよ。
ただし、リストの各要素は数値だと仮定して良い。

### ▲reduce

3-3や3-4で、組込み関数の`sum()`を紹介しました。
これは総和を返す組込み関数でした。

In [ ]:
sum([-1,-3,2,4])

総和があるならば、総乗を取るような組込み関数があるかというと、ありません。
しかし、`functools`モジュールには、総和や総乗を一般化した関数**`functool.reduce()`**があります。

`reduce()`は、第1引数にとる2引数関数を使って、第2引数を前から順に畳み込む関数です。
前から順に畳み込むとは、具体的には、第1引数が`f`で、第2引数が`[-1,-3,2,4]`のとき、`f(f(f(-1, -3), 2), 4)`という演算です。
したがって、総和も総乗も次のように表現できます。

In [ ]:
import functools
functools.reduce(lambda x,y: x+y, [-1,-3,2,4])

In [ ]:
functools.reduce(lambda x,y: x*y, [-1,-3,2,4])

`sum()`の第2引数に初期値を取れるように、`reduce()`も第3引数に初期値を取れます。

In [ ]:
sum([-1,-3,2,4], 10)

In [ ]:
functools.reduce(lambda x,y: x*y, [-1,-3,2,4], 10)

初期値は、第2引数の要素とは異なるデータ型を取ることを許されます。
与える関数の第1引数と第2引数も、異なるデータ型を取ることを許されます。
したがって、巧妙に初期値と引数関数を設定することで、様々な計算を`reduce()`で実現できます。

In [ ]:
def enumstep(x, y):
    i, ls = x
    ls.append((i,y))
    return (i + 1, ls)
functools.reduce(enumstep, 'ACDB', (0,[]))[1]

ただし、このように複雑になってくると、素直にfor文で書いた方が見やすくなることも多々あります。
`reduce()`の利用には、バランス感覚が重要です。

## イテレータ

前述の`sorted()`、`min()`、`max()`などは、リストとタプルの両方を同様に渡して処理することができます。
for文で走査（全要素を訪問）するときも、リストとタプルは同様に扱えます。
何故、異なるものを同じように扱えるのでしょうか。
それはイテレータという仕掛けがあるからです。

**イテレータ**とは、要素の集まりを走査するオブジェクトです。
組込み関数**`iter()`**によって構築し、組込み関数**`next()`**によって要素を取り出します。

In [ ]:
it = iter([1,2]) # [1,2]のイテレータを構築
next(it)

In [ ]:
next(it)

In [ ]:
next(it)

`next()`は、返す要素がないとき（走査の終了時）に、**`StopIteration`**という例外を投げます。

イテレータを`iter()`に渡すと、それ自身が返されます。

In [ ]:
it = iter([1,2]) 
it is iter(it)

イテレータもfor文で反復処理できます。

In [ ]:
it = iter([1,2])
for x in it:
    print(x)

for文では、`StopIteration`を検知して反復を自動的に終了しています。

ここで重要なのは、リストやタプルなどのデータ構造は、全てイテレータを経由して走査するということです。
つまり、リストやタプルなど異なるものから、イテレータという同様に操作できるオブジェクトを構築して利用することで、同じように走査できるようになったわけです。

In [ ]:
it = iter((1,2)) # (1,2)のイテレータを構築
for x in it:
    print(x)

ここで注意すべきことは、イテレータは1回の走査にしか使えない、使い捨てのオブジェクトだということです。
同じデータ構造を複数回走査したいときには、走査する度にイテレータを構築する必要があります。

In [ ]:
next(it) # (1,2)のイテレータitは走査が終了したまま

In [ ]:
for x in it:
    print('これは呼び出されない')

ここで憶えておくべきことは、イテレータ自体は元のデータ構造をコピーしないということです。
要素を1つ1つ訪問するという反復処理を実現するオブジェクトであり、`iter()`や`next()`は、構築元のデータ構造のサイズ（要素数）に依存しないコストで通常実装されます。
例えば、リストの先頭要素を除いた残りの部分を走査するとき、
```Python
for x in ls[1:]:
    何かの処理

```
と残りの部分をスライスとしてコピーするよりも、
```Python
it = iter(ls)
next(it) # 先頭要素を捨てる
for x in it:
    何かの処理

```
とイテレータで直接走査する方が効率的です。
これは、サイズが小さいデータ構造を扱うときには問題になりませんが、大きいものを扱うときには気を付けるべきことです。

ここまで、イテレータの使い方は、`next()`で要素を取り出すか、for文で反復するだけでしたが、実は`sorted()`、`max()`、`min()`などに渡すことができます。
文字列、タプル、リスト、辞書、イテレータなど、イテレータを介して要素を走査可能なオブジェクトを総称して**イテラブル**と呼びます。
`sorted()`・`max()`・`min()`は、イテラブルを受け取る関数です。

イテレータ`it`の中身を印字して確認したいときには、`print(*it)`と、イテレータを展開して可変長引数として`print()`を呼び出すのが簡潔で便利です。
ただし、中身を確認した後の`it`はもう利用できないこと、大量の要素を生成するイテレータには不向きであることに留意してください。

In [ ]:
it = iter(range(4))
next(it) # 先頭の 0 を捨てる
print(*it)

イテレータとイテラブルの詳細については、付録 [6-iterable](../appendix/6-iterable.ipynb) を参照してください。

### 練習
与えられたイテラブルの先頭要素を除いた残りの部分の最大値を返す関数`tailmax()`を、イテレータを使って、for文を使わずに、上の例に倣って効率的に実装せよ。

## イテレータを生成する関数

Pythonの組込み関数や標準ライブラリには、イテレータを返す関数が数多くあります。
その中には、関数を受け取る高階関数もあります。
イテレータを生成・消費する関数の適用に分解してプログラムを記述することで、イテレータを介した関数プログラミングが行えるようになります。

### map
組込み関数の**`map()`**は、第1引数に取った関数を、第2引数に取ったイテラブルの各要素に適用した結果を走査するイテレータを返します。

In [ ]:
print(*map(lambda x: x + 1, [1,-3,2,0]))

より正確には、第1引数には、$n$引数関数（$n \ge 1$）を取ることができ、第2引数以降に$n$個のイテラブルを渡すことができます。この時、一番小さい要素数に合わせて、結果のイテレータは切り詰められます。

In [ ]:
# 異なるイテラブルを受け取れる
print(*map(lambda x,y: x + y, [1,-3,2,0], (4,7,-6,5)))

In [ ]:
# 結果のイテレータが切り詰められる
print(*map(lambda x,y: x + y, range(1,10,2), range(1000000)))

`map()`とラムダ式を組み合わせるよりも、3-3で紹介した内包表現（ジェネレータ式を含む）の方が簡潔になることも少なくありません。

In [ ]:
print([x + 1 for x in  [1,-3,2,0]])  # リスト内包 

In [ ]:
print(*(x + 1 for x in  [1,-3,2,0])) # ジェネレータ式（イテレータを返す）

一方、既定義の関数を引数に渡すときには、`map()`の方が簡潔になります。
その時々で、内包表記と比べてみて、より分かりやすい方を採用しましょう。

#### 練習
第1引数で与えられた要素数まで、第2引数に与えられたイテラブルを走査するイテレータを返す関数`take()`を、for文を使わずに、`map()`と`range()`を用いて定義せよ。例えば、`take(2, 'ACDB')`は、`A` `C`を走査するイテレータを返す。

### filter
組込み関数**`filter()`**は、第1引数に単項述語（真理値を返す1引数関数）を、第2引数にイテラブルを取り、その単項述語を真にする要素だけを順に生成するイテレータを返します。

In [ ]:
print(*filter(lambda x: x % 2 == 0, range(8)))

`filter()`は、制御構造の観点で見ると、continue文によるスキップを含んだfor文に対応します。
continueを含んだfor文を使うときには、代わりに`filter()`を使うことができないか考えてみると良いでしょう。

`map()`と同様に、素直に内包表現に書き換えられます。

In [ ]:
print([x for x in range(8) if x % 2 == 0]) # リスト内包

In [ ]:
print(*(x for x in range(8) if x % 2 == 0)) # ジェネレータ式（イテレータを返す）

`filter()`とラムダ式を組み合わせるときや、`filter()`と`map()`を組み合わせるときは、内包表記を使った方が簡潔で分かりやすくなることが多いです。

#### 練習
ファイルオブジェクト（行単位のイテレータ）`f`を取って、その中の非空白行を順に生成するイテレータを返す関数`compactlines()`を、for文を使わずに、`filter()`を用いて定義せよ。

ヒント：`s`が空白文字（改行を含む）からなる文字列であるとき、`s.strip()`は空文字列を返す。

### enumerate
3-2で紹介した組込み関数の**`enumerate()`**は、実はイテレータを返します。

In [ ]:
it = enumerate('ACDB')
print(it) # リストやタプルではない

In [ ]:
print(*it)

つまり、for文や内包表現に限定されず、イテレータを消費する関数と共に使えます。

そして、`enumerate()`は、イテラブルを引数に取ります。つまり、イテレータも渡せます。
したがって、計算結果のイテレータの各要素に番号付けすることにも利用できます。

`enumerate()`の第2引数には番号付けの初期値を渡せます。

In [ ]:
print(*enumerate('ACDB',1))

`enumerate()`は、番号付けという汎用的なデータ変換を行う関数だったのです。

### ▲zip
組込み関数**`zip()`**は、`map()`の第1引数の関数が、タプル構築に固定されたものです。

In [ ]:
print(*zip(range(1,10,2), range(1000000)))

上に示したように、結果のイテレータの切り詰めも、同様に行われます。

`zip()`は、`map()`の特殊形でしかないのですが、`map()`を内包表記に書き換えるときや、結果のイテレータをfor文で反復するときに、特に役立ちます。

In [ ]:
print([x + y for x, y in zip(range(1,10,2), range(1000000))]) # リスト内包

In [ ]:
print(*(x + y for x, y in zip(range(1,10,2), range(1000000)))) # ジェネレータ式（イテレータを返す）

In [ ]:
for x, y in zip(range(1,10,2), range(1000000)): # for文で反復処理
    print(x + y)

`map()`とラムダ式を使うか、`zip()`と内包表記を使うかは、より分かりやすい方を、その時々で判断して選択しましょう。

#### 練習
イテラブルを取って、隣接要素対のイテレータを返す関数`adjpairs()`を、for文を使わずに`zip()`を使って定義せよ。
例えば、`adjsum([1,-3,2,0])`は、`(1, -3)` `(-3, 2)` `(2, 0)`を走査するイテレータを返すことになる。

### ▲reversed
組込み関数**`reversed()`**は、シーケンス（文字列、リスト、タプルなど）を受け取って、それを逆順に走査するイテレータを返します。

In [ ]:
print(*reversed('ABCD'))

In [ ]:
print(*reversed([0,1,-2,3]))

In [ ]:
print(*reversed((0,1,-2,3)))

`reversed()`は、イテレータを取れないことに留意してください。
シーケンスの詳細については、付録 iterable.ipynb を参照してください。

#### 練習
与えられたシーケンスを真ん中で折り畳んで閉じ合わせた（zipした）結果をイテレータで返す関数`clamshell()`を、for文を使わずに、`reversed()`と`take()`を使って定義せよ。
ただし、シーケンスの長さが奇数であるとき、中央の要素は結果から除外されるものとする。
例えば、`clamshell('ABCDE')`は、`(A,E)` `(B,D)`を走査するイテレータを返す。

### ▲chain
`itertools`モジュールから、特に有用なものを1つ紹介します。
**`itertools.chain()`**は、与えられた任意個のイテラブルを連結します。

In [ ]:
import itertools
# 異なるデータ型を連結したイテレータ
it = itertools.chain(range(3),       # rangeオブジェクト
                     'abc',          # 文字列
                     list('def'),    # リスト
                     tuple('ghi'),   # タプル
                     iter(range(3))) # イテレータ
print(*it)

ここで重要なのは、これら異なる5種類のデータ型に対する`+`の連結は、いずれもエラーになるということです。
`chain()`は、それぞれのイテラブルから作り出したイテレータを連結するので、データ型の違いが問題になりません。

関連して、与えられた1つのイテラブルの要素を連結する関数**`itertools.chain.from_iterable()`**もあります。

In [ ]:
print(*itertools.chain.from_iterable([range(3),         # rangeオブジェクト
                                      'abc',            # 文字列
                                      list('def'),      # リスト
                                      tuple('ghi'),     # タプル
                                      iter(range(3))])) # イテレータ

`chain()`が典型的によく使われる局面は、ファイルデータの操作です。
```Python
f = itertools.chain(open('input1'), open('input2'))
```
この`f`は、ファイル`input1`と`input2`を連結したファイルのように振る舞うイテレータです。
これは新しいファイルを作らないので、実際にファイル連結を行うよりも、大変効率が良いです。
また、
```Python
it = itertools.chain.from_iterable(f) # fはファイルのようなイテレータ
```
とすると、`f`の各行を、全て連結したような文字列のイテレータ`it`を構築できます。
これは、`''.join(f)`で文字列結合したり、（`f`がファイルオブジェクトの場合）`f.read()`でファイル全体を読み込んで文字列を構成するよりも、大変効率が良いです。

**`itertools`**モジュールは、他にも有用なイテレータ生成関数を数多く提供しています。
詳しくは、[公式ドキュメント](https://docs.python.org/ja/3/library/itertools.html)を参照してください。

## ▲**関数内関数**（クロージャ）
関数内で定義された関数（ラムダ式を含む）からは、外側のローカル変数を参照できます。

In [ ]:
def outer(x):
    def inner():
        return x
    return inner

f = outer(1)
f()

In [ ]:
g = outer(2)
g()

グローバル変数がそうであるように、外側の関数のローカル変数についても、内側の関数からは再定義が（基本的に）できません。
しかし、外側の関数では再定義できるので、注意が必要です。

In [ ]:
def outer(x):
    def inner():
        return x
    x = -x # inner()が参照する変数xを再定義
    return inner

f = outer(1)
f()

それ故に、関数を返す高階関数を記述するときには、変数定義に対してとても慎重になる必要があります。

そういう事情も含めて、関数を返す高階関数を正しく定義するのは難しいので、自分で定義せずに既存の関数を使うだけにするのが無難でしょう。

## 練習の解答

In [ ]:
ls = [('A', 1), ('B', 3), ('C', -1), ('D', 0)]
sorted(ls, key=lambda x: -x[1])

In [ ]:
max([1,3,-2,0], key=lambda x: -x)

In [ ]:
def tailmax(xs):
    it = iter(xs)
    next(it) # 先頭要素を捨てる
    return max(it)

print(tailmax([3,-4,2,1]) == 2)
print(tailmax((3,-4,2,1)) == 2)
print(tailmax('ACDC') == 'D')

In [ ]:
def take(n, xs):
    return map(lambda x, i: x, xs, range(n))

print(*take(2, 'ACDB'))

In [ ]:
def compactlines(f):
    return filter(lambda x: x.strip() != '', f)
    
fname = 'compactlines_input.txt'
fcontents = """

to be

compacted.


"""
print(fcontents, file=open(fname, 'w'))
print(list(compactlines(open(fname)))) 

In [ ]:
def adjpairs(xs):
    it = iter(xs)
    next(it) # 1つ前にずらす
    return zip(xs, it)

print(*adjpairs([1,-3,2,0]))

In [ ]:
def clamshell(xs):
    return take(len(xs)//2, zip(xs, reversed(xs)))

print(*clamshell('ABCDE'))